# Speech recognition

pip install SpeechRecognition  
Need also to install pyAudio  
Other instructions here http://people.csail.mit.edu/hubert/pyaudio/

NB: I had some troubles to install pyAudio on Ubuntu. To make it work properly use the command below:  
sudo apt-get install portaudio19-dev python-pyaudio
pip install pyAudio

https://www.codinground.com/speech-recognition/  
https://github.com/Uberi/speech_recognition/blob/master/examples/microphone_recognition.py

In [1]:
import speech_recognition as sr
import wave
import pyaudio

In [2]:
r = sr.Recognizer()

In [3]:
sr.Microphone.list_microphone_names()

['HDA Intel HDMI: 0 (hw:0,3)',
 'HDA Intel HDMI: 1 (hw:0,7)',
 'HDA Intel HDMI: 2 (hw:0,8)',
 'HDA Intel HDMI: 3 (hw:0,9)',
 'HDA Intel HDMI: 4 (hw:0,10)',
 'HDA Intel PCH: ALC3232 Analog (hw:1,0)',
 'HDA NVidia: HDMI 0 (hw:2,3)',
 'HDA NVidia: HDMI 1 (hw:2,7)',
 'HDA NVidia: HDMI 2 (hw:2,8)',
 'HDA NVidia: HDMI 3 (hw:2,9)',
 'hdmi',
 'pulse',
 'default']

In [4]:
micro = sr.Microphone(device_index=5)

In [5]:
with micro as source:
    print("Speak!")
    audio_data = r.listen(source)
    print("End!")
result = r.recognize_google(audio_data)
print (">", result)

Speak!
End!
> associate tester


In [ ]:
result

# Recording from micro

In [23]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 10
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Start Recording ...')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('... Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


## Getting the analyzed results with Google Speech recognition

In [24]:
import speech_recognition as sr
r = sr.Recognizer()

with sr.AudioFile(filename) as source:

    audio = r.record(source)
    try:
        # data = r.recognize_google(audio)
        data = r.recognize_google(audio, language="fr-FR")
        print(data)
    except:
        print("Please try again")

Histoire commence un beau matin tout le monde va bien les élèves sont heureux


# With CMUSphinx

https://cmusphinx.github.io/wiki/  
https://pypi.org/project/pocketsphinx/  
https://doc.ubuntu-fr.org/pocketsphinx

In [17]:
import sys,os
import pyaudio
import wave

hmdir = "/usr/share/pocketsphinx/model/FR/"
lmd = "/usr/share/pocketsphinx/model/FR/french3g62K.lm.dmp"
dictd = "/usr/share/pocketsphinx/model/FR/frenchWords62K.dic"
 
def decodeSpeech(hmmd,lmdir,dictp,wavfile):
    import pocketsphinx as ps
    import sphinxbase
 
    speechRec = ps.Decoder(hmm = hmmd, lm = lmdir, dict = dictp)
    wavFile = file(wavfile,'rb')
    wavFile.seek(44)
    speechRec.decode_raw(wavFile)
    result = speechRec.get_hyp()
 
    return result[0]
 
#CHUNK = 1024
CHUNK = 512
#FORMAT = pyaudio.paInt16
FORMAT = pyaudio.paALSA
CHANNELS = 1
RATE = 16000
#RATE = 44100
RECORD_SECONDS = 10
 
for x in range(10):
    fn = "o"+str(x)+".wav"
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    print("* recording")
    frames = []
    print(str(RATE / CHUNK * RECORD_SECONDS) + " size\n")
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("* done recording")
    stream.stop_stream()
    stream.close()
    wf = wave.open(fn, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    p.terminate()
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    wavfile = fn
    recognised = decodeSpeech(hmdir,lmd,dictd,wavfile)
    print (recognised)
    cm = 'espeak "'+recognised+'"'
    os.system(cm)

* recording
312.5 size

* done recording


TypeError: __init__() got an unexpected keyword argument 'hmm'